In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score,roc_curve,recall_score,roc_auc_score,auc,confusion_matrix,cohen_kappa_score, f1_score, precision_score,matthews_corrcoef 
from tqdm import tqdm
from sklearn.svm import SVC,LinearSVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectPercentile, f_classif, chi2, SelectFromModel, SelectKBest
from sklearn.feature_selection import RFE
from sklearn.preprocessing import MinMaxScaler
import scipy.stats as stats
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

# Optional imports for preprocessing and visualization
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import os

import SimpleITK as sitk
import numpy as np
from radiomics import featureextractor,imageoperations
import os
import pandas as pd
from pandas import DataFrame as DF
import warnings
import time
from time import sleep
from tqdm import tqdm
from skimage import measure

In [92]:
Data_Path= r'Test_Files'
error_file="Error_list_radiomics.txt"
Test_patients = [os.path.join(Data_Path,name) for name in os.listdir(Data_Path) if os.path.isdir(os.path.join(Data_Path, name))]
Test_patients = sorted(Test_patients)
Test_patients

['Test_Files\\LIDC-IDRI-0007',
 'Test_Files\\LIDC-IDRI-0151',
 'Test_Files\\LIDC-IDRI-0269',
 'Test_Files\\LIDC-IDRI-0423',
 'Test_Files\\LIDC-IDRI-0640',
 'Test_Files\\LIDC-IDRI-0712',
 'Test_Files\\LIDC-IDRI-0726',
 'Test_Files\\LIDC-IDRI-0863',
 'Test_Files\\LIDC-IDRI-0932',
 'Test_Files\\LIDC-IDRI-0966']

In [93]:
def Extract_Features(image,mask,params_path):
    paramsFile = os.path.abspath(params_path)
    extractor = featureextractor.RadiomicsFeatureExtractor(paramsFile)
    result = extractor.execute(image, mask)
    general_info = {'diagnostics_Configuration_EnabledImageTypes','diagnostics_Configuration_Settings',
                    'diagnostics_Image-interpolated_Maximum','diagnostics_Image-interpolated_Mean',
                    'diagnostics_Image-interpolated_Minimum','diagnostics_Image-interpolated_Size',
                    'diagnostics_Image-interpolated_Spacing','diagnostics_Image-original_Hash',
                    'diagnostics_Image-original_Maximum','diagnostics_Image-original_Mean',
                    'diagnostics_Image-original_Minimum','diagnostics_Image-original_Size',
                    'diagnostics_Image-original_Spacing','diagnostics_Mask-interpolated_BoundingBox',
                    'diagnostics_Mask-interpolated_CenterOfMass','diagnostics_Mask-interpolated_CenterOfMassIndex',
                    'diagnostics_Mask-interpolated_Maximum','diagnostics_Mask-interpolated_Mean',
                    'diagnostics_Mask-interpolated_Minimum','diagnostics_Mask-interpolated_Size',
                    'diagnostics_Mask-interpolated_Spacing','diagnostics_Mask-interpolated_VolumeNum',
                    'diagnostics_Mask-interpolated_VoxelNum','diagnostics_Mask-original_BoundingBox',
                    'diagnostics_Mask-original_CenterOfMass','diagnostics_Mask-original_CenterOfMassIndex',
                    'diagnostics_Mask-original_Hash','diagnostics_Mask-original_Size',
                    'diagnostics_Mask-original_Spacing','diagnostics_Mask-original_VolumeNum',
                    'diagnostics_Mask-original_VoxelNum','diagnostics_Versions_Numpy',
                    'diagnostics_Versions_PyRadiomics','diagnostics_Versions_PyWavelet',
                    'diagnostics_Versions_Python','diagnostics_Versions_SimpleITK',
                    'diagnostics_Image-original_Dimensionality'}
    features = dict((key, value) for key, value in result.items() if key not in general_info)
    feature_info = dict((key, value) for key, value in result.items() if key in general_info)
    return features,feature_info

In [94]:
Feature = []
test_patients = [os.path.join(Data_Path,name) for name in os.listdir(Data_Path) if os.path.isdir(os.path.join(Data_Path, name))]
test_patients = sorted(test_patients)    
for patients in tqdm(test_patients):
        #sleep(0.01)
        print(patients[-14:])
        dcm_File = patients[-14:]
        count=(len([entry for entry in os.listdir(patients) if os.path.isfile(os.path.join(patients, entry))]))
        try:

            for i in range(1,int(count/2)+1):
                roi_path = Data_Path+'/'+dcm_File+"/"+dcm_File+'_'+str(i)+'_roi.npy'
                img=np.load(roi_path)
                ROI = sitk.GetImageFromArray(img)
            
                mask_path = Data_Path+'/'+dcm_File+"/"+dcm_File+'_'+str(i)+'_label.npy'
                msk=np.load(mask_path)
                Mask = sitk.GetImageFromArray(msk)
            
                features, feature_info = Extract_Features(ROI, Mask, 'params.yaml')
                features['Patient_id']=dcm_File+'_'+str(i)+'_roi.npy'
                Feature.append(features)
        except Exception as Error:
            with open('Error_list_radiomics.txt', 'a') as error_file:
                error_file.write(f"Error: {dcm_File}:{str(Error)}\n")
                print("Error: "+dcm_File+":"+str(Error)+"\n")
            continue
    
df = DF(Feature).fillna('0')
df.to_csv(f"RM_features_Test.csv", index=False,sep=',')

  0%|          | 0/10 [00:00<?, ?it/s]

LIDC-IDRI-0007


 10%|█         | 1/10 [00:01<00:14,  1.65s/it]

LIDC-IDRI-0151


 20%|██        | 2/10 [00:02<00:08,  1.02s/it]

...

 80%|████████  | 8/10 [00:08<00:02,  1.06s/it]

LIDC-IDRI-0932


 90%|█████████ | 9/10 [00:08<00:00,  1.11it/s]

LIDC-IDRI-0966


100%|██████████| 10/10 [00:10<00:00,  1.06s/it]


In [95]:
RM_RF_model_200_features=['wavelet-LHL_glrlm_GrayLevelNonUniformity', 'log-sigma-1-0-mm-3D_gldm_DependenceNonUniformity', 'log-sigma-4-0-mm-3D_gldm_LargeDependenceEmphasis', 'log-sigma-3-0-mm-3D_firstorder_TotalEnergy', 'log-sigma-4-0-mm-3D_glszm_HighGrayLevelZoneEmphasis', 'wavelet-HHH_glrlm_RunVariance', 'log-sigma-4-0-mm-3D_glrlm_LongRunHighGrayLevelEmphasis', 'log-sigma-5-0-mm-3D_firstorder_Median', 'original_glrlm_RunEntropy', 'log-sigma-5-0-mm-3D_firstorder_Skewness', 'log-sigma-3-0-mm-3D_gldm_LargeDependenceEmphasis', 'log-sigma-5-0-mm-3D_firstorder_InterquartileRange', 'log-sigma-1-0-mm-3D_firstorder_MeanAbsoluteDeviation', 'log-sigma-1-0-mm-3D_firstorder_Energy', 'log-sigma-2-0-mm-3D_gldm_GrayLevelNonUniformity', 'log-sigma-3-0-mm-3D_gldm_DependenceNonUniformityNormalized', 'wavelet-LHL_glcm_DifferenceAverage', 'log-sigma-5-0-mm-3D_gldm_LargeDependenceEmphasis', 'log-sigma-4-0-mm-3D_firstorder_Skewness', 'log-sigma-5-0-mm-3D_firstorder_10Percentile', 'log-sigma-2-0-mm-3D_glrlm_RunLengthNonUniformity', 'log-sigma-4-0-mm-3D_firstorder_Kurtosis', 'log-sigma-2-0-mm-3D_glrlm_RunEntropy', 'log-sigma-2-0-mm-3D_firstorder_Range', 'log-sigma-3-0-mm-3D_firstorder_Minimum', 'log-sigma-4-0-mm-3D_gldm_DependenceNonUniformityNormalized', 'log-sigma-3-0-mm-3D_firstorder_Skewness', 'log-sigma-2-0-mm-3D_firstorder_MeanAbsoluteDeviation', 'log-sigma-4-0-mm-3D_firstorder_MeanAbsoluteDeviation', 'original_glrlm_RunLengthNonUniformity', 'original_glrlm_GrayLevelNonUniformity', 'log-sigma-2-0-mm-3D_firstorder_InterquartileRange', 'wavelet-LHL_glcm_Contrast', 'log-sigma-1-0-mm-3D_glrlm_RunEntropy', 'log-sigma-5-0-mm-3D_firstorder_Kurtosis', 'log-sigma-4-0-mm-3D_glrlm_RunEntropy', 'wavelet-LLH_glrlm_GrayLevelNonUniformity', 'original_shape_VoxelVolume', 'log-sigma-2-0-mm-3D_glrlm_LongRunLowGrayLevelEmphasis', 'log-sigma-5-0-mm-3D_glrlm_RunVariance', 'original_glrlm_RunVariance', 'log-sigma-4-0-mm-3D_firstorder_InterquartileRange', 'wavelet-LLL_firstorder_Minimum', 'wavelet-LHL_glcm_Id', 'original_shape_Elongation', 'log-sigma-2-0-mm-3D_firstorder_Variance', 'log-sigma-5-0-mm-3D_firstorder_Variance', 'log-sigma-3-0-mm-3D_glcm_Idn', 'wavelet-HLL_gldm_DependenceNonUniformity', 'log-sigma-5-0-mm-3D_glrlm_LongRunEmphasis', 'log-sigma-3-0-mm-3D_glszm_SizeZoneNonUniformityNormalized', 'wavelet-HHH_glrlm_LongRunEmphasis', 'log-sigma-1-0-mm-3D_firstorder_Range', 'log-sigma-3-0-mm-3D_glszm_ZoneEntropy', 'wavelet-LHL_firstorder_TotalEnergy', 'log-sigma-3-0-mm-3D_glrlm_RunLengthNonUniformity', 'log-sigma-1-0-mm-3D_firstorder_TotalEnergy', 'wavelet-HLL_firstorder_RobustMeanAbsoluteDeviation', 'log-sigma-5-0-mm-3D_glrlm_RunLengthNonUniformityNormalized', 'log-sigma-2-0-mm-3D_firstorder_RootMeanSquared', 'wavelet-LHL_glrlm_ShortRunLowGrayLevelEmphasis', 'wavelet-LLH_firstorder_TotalEnergy', 'wavelet-HHH_gldm_DependenceNonUniformity', 'log-sigma-3-0-mm-3D_firstorder_InterquartileRange', 'wavelet-LLH_glcm_DifferenceAverage', 'log-sigma-5-0-mm-3D_glrlm_LongRunHighGrayLevelEmphasis', 'wavelet-LLL_firstorder_10Percentile', 'original_firstorder_10Percentile', 'log-sigma-5-0-mm-3D_gldm_SmallDependenceLowGrayLevelEmphasis', 'log-sigma-1-0-mm-3D_glrlm_LongRunHighGrayLevelEmphasis', 'wavelet-LLL_glrlm_RunLengthNonUniformity', 'log-sigma-3-0-mm-3D_glrlm_LongRunEmphasis', 'log-sigma-4-0-mm-3D_firstorder_Maximum', 'log-sigma-5-0-mm-3D_firstorder_Entropy', 'log-sigma-5-0-mm-3D_firstorder_Maximum', 'log-sigma-5-0-mm-3D_glrlm_RunLengthNonUniformity', 'log-sigma-1-0-mm-3D_firstorder_Kurtosis', 'log-sigma-1-0-mm-3D_firstorder_RobustMeanAbsoluteDeviation', 'log-sigma-5-0-mm-3D_gldm_LargeDependenceHighGrayLevelEmphasis', 'log-sigma-1-0-mm-3D_firstorder_InterquartileRange', 'log-sigma-4-0-mm-3D_glszm_ZonePercentage', 'wavelet-LLH_glcm_Idn', 'original_glrlm_ShortRunEmphasis', 'log-sigma-1-0-mm-3D_firstorder_Variance', 'log-sigma-1-0-mm-3D_firstorder_Skewness', 'log-sigma-3-0-mm-3D_firstorder_10Percentile', 'log-sigma-5-0-mm-3D_glszm_GrayLevelNonUniformityNormalized', 'log-sigma-1-0-mm-3D_gldm_LargeDependenceHighGrayLevelEmphasis', 'log-sigma-2-0-mm-3D_gldm_DependenceNonUniformity', 'wavelet-LLH_firstorder_10Percentile', 'log-sigma-3-0-mm-3D_firstorder_RobustMeanAbsoluteDeviation', 'wavelet-LHL_glcm_Idmn', 'log-sigma-5-0-mm-3D_glcm_ClusterTendency', 'log-sigma-3-0-mm-3D_glrlm_RunEntropy', 'wavelet-LLH_firstorder_Maximum', 'log-sigma-1-0-mm-3D_firstorder_10Percentile', 'original_shape_MinorAxisLength', 'log-sigma-3-0-mm-3D_glcm_ClusterShade', 'log-sigma-5-0-mm-3D_firstorder_Range', 'log-sigma-5-0-mm-3D_glcm_Id', 'log-sigma-5-0-mm-3D_firstorder_Minimum', 'wavelet-LLL_gldm_DependenceVariance', 'log-sigma-4-0-mm-3D_glszm_GrayLevelNonUniformityNormalized', 'original_glcm_ClusterTendency', 'log-sigma-4-0-mm-3D_glcm_ClusterShade', 'original_glcm_JointEntropy', 'log-sigma-4-0-mm-3D_firstorder_Range', 'log-sigma-5-0-mm-3D_glrlm_LongRunLowGrayLevelEmphasis', 'log-sigma-5-0-mm-3D_glcm_Imc2', 'log-sigma-5-0-mm-3D_firstorder_RobustMeanAbsoluteDeviation', 'original_shape_Flatness', 'log-sigma-2-0-mm-3D_firstorder_RobustMeanAbsoluteDeviation', 'wavelet-LHL_glcm_Idn', 'log-sigma-5-0-mm-3D_glrlm_RunEntropy', 'log-sigma-2-0-mm-3D_firstorder_Maximum', 'log-sigma-5-0-mm-3D_firstorder_Mean', 'log-sigma-2-0-mm-3D_firstorder_Median', 'log-sigma-4-0-mm-3D_glszm_GrayLevelNonUniformity', 'wavelet-LLL_gldm_DependenceNonUniformityNormalized', 'log-sigma-4-0-mm-3D_glszm_SmallAreaEmphasis', 'original_shape_Maximum2DDiameterColumn', 'log-sigma-1-0-mm-3D_firstorder_RootMeanSquared', 'log-sigma-2-0-mm-3D_glszm_SmallAreaHighGrayLevelEmphasis', 'wavelet-LLL_firstorder_90Percentile', 'log-sigma-4-0-mm-3D_glcm_Idmn', 'log-sigma-1-0-mm-3D_gldm_GrayLevelVariance', 'log-sigma-2-0-mm-3D_glcm_JointEntropy', 'wavelet-LHL_glrlm_RunLengthNonUniformity', 'log-sigma-3-0-mm-3D_glcm_Imc1', 'log-sigma-2-0-mm-3D_firstorder_Minimum', 'log-sigma-5-0-mm-3D_firstorder_RootMeanSquared', 'log-sigma-5-0-mm-3D_firstorder_TotalEnergy', 'log-sigma-5-0-mm-3D_glszm_GrayLevelVariance', 'log-sigma-1-0-mm-3D_firstorder_Maximum', 'log-sigma-3-0-mm-3D_glcm_DifferenceVariance', 'log-sigma-4-0-mm-3D_glcm_Idn', 'log-sigma-2-0-mm-3D_gldm_DependenceVariance', 'log-sigma-3-0-mm-3D_firstorder_Mean', 'log-sigma-3-0-mm-3D_glcm_Imc2', 'log-sigma-5-0-mm-3D_gldm_SmallDependenceHighGrayLevelEmphasis', 'log-sigma-5-0-mm-3D_gldm_DependenceNonUniformityNormalized', 'wavelet-HLL_firstorder_10Percentile', 'log-sigma-3-0-mm-3D_gldm_DependenceVariance', 'log-sigma-4-0-mm-3D_glszm_LargeAreaHighGrayLevelEmphasis', 'log-sigma-5-0-mm-3D_firstorder_MeanAbsoluteDeviation', 'wavelet-HLL_glcm_SumSquares', 'log-sigma-2-0-mm-3D_glcm_DifferenceAverage', 'log-sigma-3-0-mm-3D_gldm_LargeDependenceHighGrayLevelEmphasis', 'wavelet-LLH_firstorder_Minimum', 'wavelet-HLL_firstorder_Minimum', 'log-sigma-1-0-mm-3D_firstorder_Entropy', 'log-sigma-3-0-mm-3D_glcm_DifferenceEntropy', 'log-sigma-1-0-mm-3D_glrlm_GrayLevelVariance', 'wavelet-LLL_glrlm_LongRunEmphasis', 'log-sigma-4-0-mm-3D_glszm_SmallAreaHighGrayLevelEmphasis', 'wavelet-LLL_firstorder_Kurtosis', 'log-sigma-4-0-mm-3D_glrlm_LongRunLowGrayLevelEmphasis', 'wavelet-LHL_glcm_Idm', 'log-sigma-5-0-mm-3D_glrlm_GrayLevelNonUniformityNormalized', 'original_firstorder_Minimum', 'log-sigma-3-0-mm-3D_firstorder_Kurtosis', 'original_firstorder_Variance', 'wavelet-HHL_gldm_GrayLevelNonUniformity', 'wavelet-LHL_glcm_Correlation', 'log-sigma-1-0-mm-3D_firstorder_Minimum', 'wavelet-LLL_gldm_LargeDependenceHighGrayLevelEmphasis', 'log-sigma-4-0-mm-3D_firstorder_Uniformity', 'log-sigma-1-0-mm-3D_glrlm_ShortRunLowGrayLevelEmphasis', 'wavelet-LHL_glcm_SumSquares', 'wavelet-LLH_glcm_Idm', 'wavelet-LLH_glcm_InverseVariance', 'wavelet-HLL_firstorder_MeanAbsoluteDeviation', 'original_firstorder_Energy', 'log-sigma-1-0-mm-3D_glcm_SumEntropy', 'wavelet-LLL_glcm_Imc2', 'log-sigma-1-0-mm-3D_glcm_Imc1', 'original_firstorder_Mean', 'log-sigma-5-0-mm-3D_glcm_ClusterProminence', 'log-sigma-5-0-mm-3D_glcm_Imc1', 'wavelet-HHH_firstorder_TotalEnergy', 'wavelet-LLL_firstorder_Variance', 'log-sigma-4-0-mm-3D_glcm_InverseVariance', 'log-sigma-4-0-mm-3D_firstorder_10Percentile', 'wavelet-LLH_glcm_Imc1', 'log-sigma-2-0-mm-3D_glszm_LargeAreaLowGrayLevelEmphasis', 'wavelet-HHL_glrlm_LongRunLowGrayLevelEmphasis', 'log-sigma-2-0-mm-3D_glcm_Correlation', 'log-sigma-2-0-mm-3D_glrlm_RunPercentage', 'log-sigma-4-0-mm-3D_firstorder_Minimum', 'original_firstorder_Range', 'log-sigma-1-0-mm-3D_gldm_GrayLevelNonUniformity', 'log-sigma-3-0-mm-3D_firstorder_MeanAbsoluteDeviation', 'wavelet-LLH_glcm_Correlation', 'log-sigma-2-0-mm-3D_gldm_SmallDependenceEmphasis', 'log-sigma-4-0-mm-3D_gldm_DependenceNonUniformity', 'log-sigma-3-0-mm-3D_firstorder_RootMeanSquared', 'log-sigma-2-0-mm-3D_firstorder_Energy', 'log-sigma-4-0-mm-3D_glrlm_RunPercentage', 'log-sigma-4-0-mm-3D_glcm_JointEnergy', 'log-sigma-4-0-mm-3D_firstorder_RobustMeanAbsoluteDeviation']
RM_RF_model_100_features=RM_RF_model_200_features[:100]


In [96]:
Test_DF=pd.read_csv("./RM_features_Test.csv")
scaler = StandardScaler()

X_1218=Test_DF.drop('Patient_id', axis=1)
#X_test = scaler.fit_transform(X_1218)
#X_1218=X_test
X_200=Test_DF[RM_RF_model_200_features]
X_100=Test_DF[RM_RF_model_100_features]

In [97]:
import joblib
models={
 
    }

In [98]:
import os
import shutil
import csv

def copy_files_with_suffix(source_dir, destination_dir_image,  image_suffix):
    # Ensure the destination directory exists
    os.makedirs(destination_dir_image, exist_ok=True)
    
    # Iterate through all files in the source directory
    for root, dirs, files in os.walk(source_dir):
        for file in files:
            # Check if the file name ends with the specified suffix
            print(file.split("_"))
            if file[-8:]==image_suffix:
                source_image_path = os.path.join(root, file)
                destination_image_path = os.path.join(destination_dir_image, file)
                # Copy the file to the destination directory
                shutil.copy2(source_image_path, destination_image_path)
                print(f"Copied: {file} from {source_image_path} to {destination_image_path}")
               
            

# Replace these paths and pattern with your actual values
source_directory = './Test_Files'
destination_directory_image = './ROI_ny_files'

image_name_suffix = '_roi.npy'


copy_files_with_suffix(source_directory, destination_directory_image, image_name_suffix)

['LIDC-IDRI-0007', '1', 'label.npy']
['LIDC-IDRI-0007', '1', 'roi.npy']
Copied: LIDC-IDRI-0007_1_roi.npy from ./Test_Files\LIDC-IDRI-0007\LIDC-IDRI-0007_1_roi.npy to ./ROI_ny_files\LIDC-IDRI-0007_1_roi.npy
['LIDC-IDRI-0151', '1', 'label.npy']
['LIDC-IDRI-0151', '1', 'roi.npy']
Copied: LIDC-IDRI-0151_1_roi.npy from ./Test_Files\LIDC-IDRI-0151\LIDC-IDRI-0151_1_roi.npy to ./ROI_ny_files\LIDC-IDRI-0151_1_roi.npy
['LIDC-IDRI-0269', '1', 'label.npy']
['LIDC-IDRI-0269', '1', 'roi.npy']
Copied: LIDC-IDRI-0269_1_roi.npy from ./Test_Files\LIDC-IDRI-0269\LIDC-IDRI-0269_1_roi.npy to ./ROI_ny_files\LIDC-IDRI-0269_1_roi.npy
['LIDC-IDRI-0423', '1', 'label.npy']
['LIDC-IDRI-0423', '1', 'roi.npy']
Copied: LIDC-IDRI-0423_1_roi.npy from ./Test_Files\LIDC-IDRI-0423\LIDC-IDRI-0423_1_roi.npy to ./ROI_ny_files\LIDC-IDRI-0423_1_roi.npy
['LIDC-IDRI-0640', '1', 'label.npy']
['LIDC-IDRI-0640', '1', 'roi.npy']
Copied: LIDC-IDRI-0640_1_roi.npy from ./Test_Files\LIDC-IDRI-0640\LIDC-IDRI-0640_1_roi.npy to ./ROI_ny_f

In [99]:
import tensorflow as tf
import os
from tensorflow.keras.models import load_model
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Load the model
model = load_model('feature_extractor_model_27-05-2024_128.h5')

# Load or prepare your test data (replace this with your actual test data)
# X_test = ... 
# y_test = ...
train_data_path = r'./ROI_ny_files'
train_patients = [name for name in os.listdir(train_data_path) if os.path.isfile(os.path.join(train_data_path, name))]
file_list = sorted(train_patients)
data = [np.expand_dims(np.load(f'{train_data_path}/{file}'), axis=-1) for file in file_list]
X_test = np.array(data)
# Ensure X_test is a NumPy array
# X_test = np.array(X_test)

# Make predictions
predictions = model.predict(X_test)

predictions_df = pd.DataFrame(predictions)

# Save to CSV
predictions_df.to_csv('Deep_Learning_Test', index=False)
# For a classification model, you might need to convert predictions to class labels



1/1 [==============================] - 3s 3s/step


In [100]:
predictions_df=pd.read_csv('Deep_Learning_Test')

In [101]:
DL_20_top_feature_names=['85', '105', '63', '118', '84', '20', '30', '32', '45', '127', '83', '107', '36', '5', '61', '103', '35', '106', '57', '81']
DL_10_top_feature_names=['85', '105', '63', '118', '84', '20', '30', '32', '45', '127']

RM_200_top_feature_names=['log-sigma-5-0-mm-3D_firstorder_Range', 'wavelet-HLH_firstorder_TotalEnergy', 'log-sigma-2-0-mm-3D_glrlm_RunEntropy', 'original_shape_Maximum2DDiameterColumn', 'log-sigma-5-0-mm-3D_glrlm_RunVariance', 'log-sigma-4-0-mm-3D_firstorder_Skewness', 'wavelet-LLH_glrlm_ShortRunHighGrayLevelEmphasis', 'wavelet-LHL_gldm_DependenceNonUniformity', 'log-sigma-5-0-mm-3D_firstorder_InterquartileRange', 'original_shape_LeastAxisLength', 'log-sigma-3-0-mm-3D_firstorder_TotalEnergy', 'log-sigma-5-0-mm-3D_firstorder_MeanAbsoluteDeviation', 'log-sigma-3-0-mm-3D_glszm_LowGrayLevelZoneEmphasis', 'wavelet-LLH_firstorder_TotalEnergy', 'log-sigma-3-0-mm-3D_firstorder_Range', 'original_glrlm_GrayLevelNonUniformity', 'log-sigma-2-0-mm-3D_firstorder_TotalEnergy', 'wavelet-LHL_firstorder_TotalEnergy', 'log-sigma-2-0-mm-3D_gldm_LargeDependenceEmphasis', 'original_glrlm_RunEntropy', 'log-sigma-4-0-mm-3D_firstorder_RootMeanSquared', 'wavelet-LHL_glcm_DifferenceEntropy', 'wavelet-LHH_firstorder_TotalEnergy', 'log-sigma-4-0-mm-3D_glrlm_GrayLevelNonUniformity', 'log-sigma-2-0-mm-3D_firstorder_RobustMeanAbsoluteDeviation', 'log-sigma-4-0-mm-3D_glcm_Imc1', 'log-sigma-3-0-mm-3D_gldm_LargeDependenceEmphasis', 'log-sigma-2-0-mm-3D_firstorder_Skewness', 'log-sigma-1-0-mm-3D_firstorder_Energy', 'wavelet-LHL_glcm_Idmn', 'wavelet-LLL_firstorder_Minimum', 'wavelet-LLH_firstorder_Energy', 'log-sigma-5-0-mm-3D_glrlm_RunEntropy', 'wavelet-HLL_firstorder_MeanAbsoluteDeviation', 'log-sigma-3-0-mm-3D_glszm_HighGrayLevelZoneEmphasis', 'log-sigma-5-0-mm-3D_glszm_HighGrayLevelZoneEmphasis', 'log-sigma-2-0-mm-3D_glszm_GrayLevelNonUniformityNormalized', 'log-sigma-5-0-mm-3D_firstorder_Minimum', 'log-sigma-4-0-mm-3D_firstorder_RobustMeanAbsoluteDeviation', 'log-sigma-5-0-mm-3D_firstorder_RobustMeanAbsoluteDeviation', 'wavelet-LLL_glrlm_LongRunHighGrayLevelEmphasis', 'wavelet-HHH_glrlm_RunLengthNonUniformity', 'log-sigma-3-0-mm-3D_firstorder_Median', 'log-sigma-5-0-mm-3D_glrlm_RunLengthNonUniformityNormalized', 'log-sigma-5-0-mm-3D_firstorder_Kurtosis', 'wavelet-HHH_glrlm_LongRunLowGrayLevelEmphasis', 'log-sigma-5-0-mm-3D_glcm_Idm', 'wavelet-LLH_glrlm_ShortRunEmphasis', 'log-sigma-5-0-mm-3D_gldm_DependenceNonUniformityNormalized', 'original_shape_Maximum2DDiameterRow', 'wavelet-LLL_firstorder_Maximum', 'log-sigma-2-0-mm-3D_firstorder_10Percentile', 'original_shape_Flatness', 'log-sigma-5-0-mm-3D_firstorder_Variance', 'original_firstorder_TotalEnergy', 'log-sigma-2-0-mm-3D_firstorder_Energy', 'log-sigma-3-0-mm-3D_firstorder_90Percentile', 'original_shape_SurfaceVolumeRatio', 'log-sigma-3-0-mm-3D_firstorder_MeanAbsoluteDeviation', 'wavelet-LHL_glcm_Contrast', 'log-sigma-4-0-mm-3D_glszm_LargeAreaEmphasis', 'log-sigma-1-0-mm-3D_firstorder_Skewness', 'wavelet-HLL_glrlm_GrayLevelNonUniformity', 'wavelet-LHH_glszm_ZonePercentage', 'log-sigma-5-0-mm-3D_gldm_LargeDependenceEmphasis', 'wavelet-LLL_gldm_DependenceNonUniformity', 'log-sigma-4-0-mm-3D_firstorder_MeanAbsoluteDeviation', 'log-sigma-5-0-mm-3D_gldm_DependenceNonUniformity', 'log-sigma-2-0-mm-3D_glrlm_GrayLevelNonUniformityNormalized', 'wavelet-LLH_glcm_DifferenceAverage', 'wavelet-LLL_gldm_GrayLevelNonUniformity', 'log-sigma-3-0-mm-3D_firstorder_RootMeanSquared', 'wavelet-HLL_firstorder_Mean', 'log-sigma-4-0-mm-3D_glrlm_LongRunLowGrayLevelEmphasis', 'log-sigma-3-0-mm-3D_firstorder_Kurtosis', 'log-sigma-4-0-mm-3D_glcm_JointEnergy', 'original_shape_Maximum2DDiameterSlice', 'log-sigma-1-0-mm-3D_firstorder_90Percentile', 'log-sigma-1-0-mm-3D_firstorder_Kurtosis', 'wavelet-HLL_firstorder_InterquartileRange', 'wavelet-LLH_glrlm_LongRunHighGrayLevelEmphasis', 'log-sigma-2-0-mm-3D_firstorder_InterquartileRange', 'log-sigma-3-0-mm-3D_glszm_SmallAreaEmphasis', 'log-sigma-4-0-mm-3D_glrlm_LongRunEmphasis', 'log-sigma-2-0-mm-3D_firstorder_Median', 'log-sigma-4-0-mm-3D_glszm_GrayLevelNonUniformity', 'wavelet-HLL_firstorder_Energy', 'log-sigma-4-0-mm-3D_glcm_Correlation', 'original_firstorder_Minimum', 'wavelet-LHH_gldm_DependenceNonUniformity', 'wavelet-LLH_glcm_Idm', 'log-sigma-5-0-mm-3D_firstorder_Skewness', 'wavelet-LLH_glcm_Imc2', 'wavelet-LHL_glcm_MaximumProbability', 'wavelet-HHH_glrlm_GrayLevelNonUniformity', 'wavelet-HLH_glrlm_GrayLevelNonUniformity', 'log-sigma-4-0-mm-3D_firstorder_Median', 'wavelet-LLL_firstorder_Kurtosis', 'log-sigma-5-0-mm-3D_firstorder_90Percentile', 'log-sigma-4-0-mm-3D_firstorder_Kurtosis', 'wavelet-LHL_firstorder_Energy', 'log-sigma-3-0-mm-3D_glcm_DifferenceEntropy', 'wavelet-LLL_firstorder_Energy', 'log-sigma-5-0-mm-3D_glcm_MaximumProbability', 'log-sigma-4-0-mm-3D_firstorder_10Percentile', 'log-sigma-1-0-mm-3D_firstorder_RootMeanSquared', 'wavelet-LHL_glrlm_RunLengthNonUniformity', 'wavelet-LLH_glcm_Idn', 'log-sigma-5-0-mm-3D_glszm_GrayLevelVariance', 'original_glcm_Contrast', 'wavelet-HLL_firstorder_10Percentile', 'log-sigma-3-0-mm-3D_firstorder_Skewness', 'log-sigma-3-0-mm-3D_gldm_DependenceEntropy', 'wavelet-LLL_gldm_HighGrayLevelEmphasis', 'wavelet-HLL_firstorder_TotalEnergy', 'log-sigma-3-0-mm-3D_firstorder_Variance', 'log-sigma-3-0-mm-3D_glrlm_ShortRunHighGrayLevelEmphasis', 'log-sigma-5-0-mm-3D_glrlm_GrayLevelVariance', 'wavelet-LLL_firstorder_Variance', 'log-sigma-4-0-mm-3D_glszm_HighGrayLevelZoneEmphasis', 'wavelet-LLL_firstorder_TotalEnergy', 'log-sigma-1-0-mm-3D_firstorder_Maximum', 'log-sigma-4-0-mm-3D_glrlm_RunPercentage', 'log-sigma-1-0-mm-3D_glrlm_RunVariance', 'log-sigma-5-0-mm-3D_glszm_ZonePercentage', 'wavelet-LLH_glcm_InverseVariance', 'log-sigma-2-0-mm-3D_firstorder_MeanAbsoluteDeviation', 'wavelet-LHL_glcm_Idn', 'wavelet-LLH_firstorder_Range', 'log-sigma-1-0-mm-3D_glrlm_GrayLevelNonUniformityNormalized', 'wavelet-HLL_firstorder_Range', 'log-sigma-4-0-mm-3D_firstorder_Mean', 'log-sigma-5-0-mm-3D_firstorder_Maximum', 'original_firstorder_Range', 'log-sigma-4-0-mm-3D_glrlm_RunVariance', 'log-sigma-5-0-mm-3D_glszm_LargeAreaHighGrayLevelEmphasis', 'log-sigma-1-0-mm-3D_gldm_LargeDependenceLowGrayLevelEmphasis', 'log-sigma-2-0-mm-3D_glszm_SizeZoneNonUniformityNormalized', 'wavelet-LLH_firstorder_RobustMeanAbsoluteDeviation', 'log-sigma-5-0-mm-3D_glcm_Imc1', 'wavelet-LLH_glcm_Correlation', 'log-sigma-5-0-mm-3D_gldm_GrayLevelVariance', 'wavelet-LLL_glrlm_ShortRunLowGrayLevelEmphasis', 'log-sigma-4-0-mm-3D_gldm_SmallDependenceEmphasis', 'wavelet-HHH_glrlm_LongRunEmphasis', 'wavelet-LLL_firstorder_Range', 'log-sigma-3-0-mm-3D_firstorder_10Percentile', 'log-sigma-5-0-mm-3D_firstorder_Median', 'log-sigma-5-0-mm-3D_glcm_ClusterShade', 'log-sigma-1-0-mm-3D_gldm_LowGrayLevelEmphasis', 'original_firstorder_MeanAbsoluteDeviation', 'original_glrlm_ShortRunHighGrayLevelEmphasis', 'log-sigma-3-0-mm-3D_glcm_ClusterShade', 'log-sigma-5-0-mm-3D_glrlm_LongRunHighGrayLevelEmphasis', 'log-sigma-4-0-mm-3D_glszm_ZonePercentage', 'log-sigma-1-0-mm-3D_firstorder_Range', 'wavelet-LLL_firstorder_Skewness', 'log-sigma-3-0-mm-3D_firstorder_Mean', 'original_shape_Maximum3DDiameter', 'log-sigma-4-0-mm-3D_firstorder_InterquartileRange', 'original_shape_MinorAxisLength', 'wavelet-LLL_glrlm_HighGrayLevelRunEmphasis', 'wavelet-LLL_glrlm_ShortRunHighGrayLevelEmphasis', 'wavelet-LHL_gldm_GrayLevelNonUniformity', 'log-sigma-5-0-mm-3D_glrlm_LongRunLowGrayLevelEmphasis', 'log-sigma-3-0-mm-3D_firstorder_Minimum', 'log-sigma-5-0-mm-3D_gldm_SmallDependenceHighGrayLevelEmphasis', 'log-sigma-2-0-mm-3D_glrlm_RunLengthNonUniformityNormalized', 'log-sigma-2-0-mm-3D_firstorder_Variance', 'log-sigma-4-0-mm-3D_firstorder_Range', 'log-sigma-3-0-mm-3D_glcm_ClusterTendency', 'original_glcm_ClusterTendency', 'log-sigma-1-0-mm-3D_glszm_LargeAreaHighGrayLevelEmphasis', 'wavelet-HLL_firstorder_Variance', 'wavelet-LLH_firstorder_Maximum', 'log-sigma-3-0-mm-3D_glrlm_LongRunEmphasis', 'wavelet-LLH_firstorder_MeanAbsoluteDeviation', 'log-sigma-4-0-mm-3D_glrlm_GrayLevelNonUniformityNormalized', 'log-sigma-5-0-mm-3D_glszm_GrayLevelNonUniformity', 'log-sigma-1-0-mm-3D_firstorder_MeanAbsoluteDeviation', 'log-sigma-4-0-mm-3D_glszm_LargeAreaHighGrayLevelEmphasis', 'wavelet-LLH_gldm_GrayLevelNonUniformity', 'log-sigma-5-0-mm-3D_glcm_SumSquares', 'log-sigma-1-0-mm-3D_glrlm_LongRunHighGrayLevelEmphasis', 'log-sigma-4-0-mm-3D_gldm_LargeDependenceEmphasis', 'wavelet-LHL_glcm_DifferenceVariance', 'wavelet-LHL_glcm_ClusterProminence', 'log-sigma-5-0-mm-3D_glcm_InverseVariance', 'original_glrlm_ShortRunLowGrayLevelEmphasis', 'log-sigma-2-0-mm-3D_glrlm_HighGrayLevelRunEmphasis', 'log-sigma-1-0-mm-3D_glszm_LargeAreaEmphasis', 'log-sigma-1-0-mm-3D_firstorder_Minimum', 'log-sigma-1-0-mm-3D_glcm_ClusterTendency', 'log-sigma-3-0-mm-3D_firstorder_InterquartileRange', 'log-sigma-1-0-mm-3D_glcm_Idm', 'wavelet-HLH_gldm_GrayLevelNonUniformity', 'log-sigma-3-0-mm-3D_glrlm_GrayLevelVariance', 'original_glcm_SumEntropy', 'wavelet-LHL_glcm_Id', 'log-sigma-1-0-mm-3D_gldm_GrayLevelVariance']
RM_100_top_feature_names=['log-sigma-5-0-mm-3D_firstorder_Range', 'wavelet-HLH_firstorder_TotalEnergy', 'log-sigma-2-0-mm-3D_glrlm_RunEntropy', 'original_shape_Maximum2DDiameterColumn', 'log-sigma-5-0-mm-3D_glrlm_RunVariance', 'log-sigma-4-0-mm-3D_firstorder_Skewness', 'wavelet-LLH_glrlm_ShortRunHighGrayLevelEmphasis', 'wavelet-LHL_gldm_DependenceNonUniformity', 'log-sigma-5-0-mm-3D_firstorder_InterquartileRange', 'original_shape_LeastAxisLength', 'log-sigma-3-0-mm-3D_firstorder_TotalEnergy', 'log-sigma-5-0-mm-3D_firstorder_MeanAbsoluteDeviation', 'log-sigma-3-0-mm-3D_glszm_LowGrayLevelZoneEmphasis', 'wavelet-LLH_firstorder_TotalEnergy', 'log-sigma-3-0-mm-3D_firstorder_Range', 'original_glrlm_GrayLevelNonUniformity', 'log-sigma-2-0-mm-3D_firstorder_TotalEnergy', 'wavelet-LHL_firstorder_TotalEnergy', 'log-sigma-2-0-mm-3D_gldm_LargeDependenceEmphasis', 'original_glrlm_RunEntropy', 'log-sigma-4-0-mm-3D_firstorder_RootMeanSquared', 'wavelet-LHL_glcm_DifferenceEntropy', 'wavelet-LHH_firstorder_TotalEnergy', 'log-sigma-4-0-mm-3D_glrlm_GrayLevelNonUniformity', 'log-sigma-2-0-mm-3D_firstorder_RobustMeanAbsoluteDeviation', 'log-sigma-4-0-mm-3D_glcm_Imc1', 'log-sigma-3-0-mm-3D_gldm_LargeDependenceEmphasis', 'log-sigma-2-0-mm-3D_firstorder_Skewness', 'log-sigma-1-0-mm-3D_firstorder_Energy', 'wavelet-LHL_glcm_Idmn', 'wavelet-LLL_firstorder_Minimum', 'wavelet-LLH_firstorder_Energy', 'log-sigma-5-0-mm-3D_glrlm_RunEntropy', 'wavelet-HLL_firstorder_MeanAbsoluteDeviation', 'log-sigma-3-0-mm-3D_glszm_HighGrayLevelZoneEmphasis', 'log-sigma-5-0-mm-3D_glszm_HighGrayLevelZoneEmphasis', 'log-sigma-2-0-mm-3D_glszm_GrayLevelNonUniformityNormalized', 'log-sigma-5-0-mm-3D_firstorder_Minimum', 'log-sigma-4-0-mm-3D_firstorder_RobustMeanAbsoluteDeviation', 'log-sigma-5-0-mm-3D_firstorder_RobustMeanAbsoluteDeviation', 'wavelet-LLL_glrlm_LongRunHighGrayLevelEmphasis', 'wavelet-HHH_glrlm_RunLengthNonUniformity', 'log-sigma-3-0-mm-3D_firstorder_Median', 'log-sigma-5-0-mm-3D_glrlm_RunLengthNonUniformityNormalized', 'log-sigma-5-0-mm-3D_firstorder_Kurtosis', 'wavelet-HHH_glrlm_LongRunLowGrayLevelEmphasis', 'log-sigma-5-0-mm-3D_glcm_Idm', 'wavelet-LLH_glrlm_ShortRunEmphasis', 'log-sigma-5-0-mm-3D_gldm_DependenceNonUniformityNormalized', 'original_shape_Maximum2DDiameterRow', 'wavelet-LLL_firstorder_Maximum', 'log-sigma-2-0-mm-3D_firstorder_10Percentile', 'original_shape_Flatness', 'log-sigma-5-0-mm-3D_firstorder_Variance', 'original_firstorder_TotalEnergy', 'log-sigma-2-0-mm-3D_firstorder_Energy', 'log-sigma-3-0-mm-3D_firstorder_90Percentile', 'original_shape_SurfaceVolumeRatio', 'log-sigma-3-0-mm-3D_firstorder_MeanAbsoluteDeviation', 'wavelet-LHL_glcm_Contrast', 'log-sigma-4-0-mm-3D_glszm_LargeAreaEmphasis', 'log-sigma-1-0-mm-3D_firstorder_Skewness', 'wavelet-HLL_glrlm_GrayLevelNonUniformity', 'wavelet-LHH_glszm_ZonePercentage', 'log-sigma-5-0-mm-3D_gldm_LargeDependenceEmphasis', 'wavelet-LLL_gldm_DependenceNonUniformity', 'log-sigma-4-0-mm-3D_firstorder_MeanAbsoluteDeviation', 'log-sigma-5-0-mm-3D_gldm_DependenceNonUniformity', 'log-sigma-2-0-mm-3D_glrlm_GrayLevelNonUniformityNormalized', 'wavelet-LLH_glcm_DifferenceAverage', 'wavelet-LLL_gldm_GrayLevelNonUniformity', 'log-sigma-3-0-mm-3D_firstorder_RootMeanSquared', 'wavelet-HLL_firstorder_Mean', 'log-sigma-4-0-mm-3D_glrlm_LongRunLowGrayLevelEmphasis', 'log-sigma-3-0-mm-3D_firstorder_Kurtosis', 'log-sigma-4-0-mm-3D_glcm_JointEnergy', 'original_shape_Maximum2DDiameterSlice', 'log-sigma-1-0-mm-3D_firstorder_90Percentile', 'log-sigma-1-0-mm-3D_firstorder_Kurtosis', 'wavelet-HLL_firstorder_InterquartileRange', 'wavelet-LLH_glrlm_LongRunHighGrayLevelEmphasis', 'log-sigma-2-0-mm-3D_firstorder_InterquartileRange', 'log-sigma-3-0-mm-3D_glszm_SmallAreaEmphasis', 'log-sigma-4-0-mm-3D_glrlm_LongRunEmphasis', 'log-sigma-2-0-mm-3D_firstorder_Median', 'log-sigma-4-0-mm-3D_glszm_GrayLevelNonUniformity', 'wavelet-HLL_firstorder_Energy', 'log-sigma-4-0-mm-3D_glcm_Correlation', 'original_firstorder_Minimum', 'wavelet-LHH_gldm_DependenceNonUniformity', 'wavelet-LLH_glcm_Idm', 'log-sigma-5-0-mm-3D_firstorder_Skewness', 'wavelet-LLH_glcm_Imc2', 'wavelet-LHL_glcm_MaximumProbability', 'wavelet-HHH_glrlm_GrayLevelNonUniformity', 'wavelet-HLH_glrlm_GrayLevelNonUniformity', 'log-sigma-4-0-mm-3D_firstorder_Median', 'wavelet-LLL_firstorder_Kurtosis', 'log-sigma-5-0-mm-3D_firstorder_90Percentile', 'log-sigma-4-0-mm-3D_firstorder_Kurtosis']

In [102]:
X_1218=Test_DF.drop('Patient_id', axis=1)
X_200=Test_DF[RM_200_top_feature_names]
X_100=Test_DF[RM_200_top_feature_names]




In [103]:
X_128=predictions_df

X_20=predictions_df[DL_20_top_feature_names]
X_10=predictions_df[DL_10_top_feature_names]

In [110]:
models = {
    'Radiomics_1218_Features': [joblib.load('./RM_RF_1218_features.pkl'), X_1218],
    'Radiomics_200_Features': [joblib.load('./RM_RF_200_features.pkl'), X_200],
    'Radiomics_100_Features': [joblib.load('./RM_RF_100_features.pkl'), X_100],
    'Fuse_1218_128_Features': [joblib.load('./Fuse_1218_128_features.pkl'), pd.concat([X_1218, X_128], axis=1)],
    'Fuse_1218_20_Features': [joblib.load('./Fuse_1218_20_features.pkl'), pd.concat([X_1218, X_20], axis=1)],
    'Fuse_1218_10_Features': [joblib.load('./Fuse_1218_10_features.pkl'), pd.concat([X_1218, X_10], axis=1)],
    'Fuse_200_128_Features': [joblib.load('./Fuse_200_128_features.pkl'), pd.concat([X_200, X_128], axis=1)],
    'Fuse_200_20_Features': [joblib.load('./Fuse_200_20_features.pkl'), pd.concat([X_200, X_20], axis=1)],
    'Fuse_200_10_Features': [joblib.load('./Fuse_200_10_features.pkl'), pd.concat([X_200, X_10], axis=1)],
    'Fuse_100_128_Features': [joblib.load('./Fuse_100_10_features.pkl'), pd.concat([X_100, X_128], axis=1)],
    'Fuse_100_20_Features': [joblib.load('./Fuse_100_20_features.pkl'), pd.concat([X_100, X_20], axis=1)],
    'Fuse_100_10_Features': [joblib.load('./Fuse_100_10_features.pkl'), pd.concat([X_100, X_10], axis=1)]
}

In [111]:
for name, model_def in models.items():
    print(f"Training {name}...")
    model=model_def[0]
    X_test=model_def[1]
    #print(X_test.columns)
    # scaler = StandardScaler()
    # X_test = scaler.fit_transform(X_test)
# Make predictions on the test set
    y_pred_best = model.predict(X_test)
    # Evaluate the performance of the model using appropriate metrics

    print(y_pred_best)
    y_test_p=pd.DataFrame(y_pred_best)
    y_test_p['Image File']=Test_DF['Patient_id']
    Pr_file=f"Prediction_{name}.csv"
    y_test_p.to_csv(Pr_file)

Training Radiomics_1218_Features...
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Training Radiomics_200_Features...
[1 1 0 0 0 1 0 0 0 1 0 0 0 0 0]
Training Fuse_1218_128_Features...
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Training Fuse_1218_20_Features...
[1 1 0 0 0 1 0 0 0 1 0 0 0 0 0]
Training Fuse_1218_10_Features...
[1 1 0 0 0 1 0 0 0 1 0 0 0 0 0]
Training Fuse_200_128_Features...
[1 1 0 0 0 1 0 0 0 1 0 0 0 0 0]
Training Fuse_200_20_Features...
[1 1 0 0 0 1 0 0 0 1 0 0 0 0 0]
Training Fuse_200_10_Features...
[1 1 0 0 0 1 0 0 0 1 0 0 0 0 0]


In [8]:
RM_RF_model_200_features=['wavelet-LHL_glrlm_GrayLevelNonUniformity', 'log-sigma-1-0-mm-3D_gldm_DependenceNonUniformity', 'log-sigma-4-0-mm-3D_gldm_LargeDependenceEmphasis', 'log-sigma-3-0-mm-3D_firstorder_TotalEnergy', 'log-sigma-4-0-mm-3D_glszm_HighGrayLevelZoneEmphasis', 'wavelet-HHH_glrlm_RunVariance', 'log-sigma-4-0-mm-3D_glrlm_LongRunHighGrayLevelEmphasis', 'log-sigma-5-0-mm-3D_firstorder_Median', 'original_glrlm_RunEntropy', 'log-sigma-5-0-mm-3D_firstorder_Skewness', 'log-sigma-3-0-mm-3D_gldm_LargeDependenceEmphasis', 'log-sigma-5-0-mm-3D_firstorder_InterquartileRange', 'log-sigma-1-0-mm-3D_firstorder_MeanAbsoluteDeviation', 'log-sigma-1-0-mm-3D_firstorder_Energy', 'log-sigma-2-0-mm-3D_gldm_GrayLevelNonUniformity', 'log-sigma-3-0-mm-3D_gldm_DependenceNonUniformityNormalized', 'wavelet-LHL_glcm_DifferenceAverage', 'log-sigma-5-0-mm-3D_gldm_LargeDependenceEmphasis', 'log-sigma-4-0-mm-3D_firstorder_Skewness', 'log-sigma-5-0-mm-3D_firstorder_10Percentile', 'log-sigma-2-0-mm-3D_glrlm_RunLengthNonUniformity', 'log-sigma-4-0-mm-3D_firstorder_Kurtosis', 'log-sigma-2-0-mm-3D_glrlm_RunEntropy', 'log-sigma-2-0-mm-3D_firstorder_Range', 'log-sigma-3-0-mm-3D_firstorder_Minimum', 'log-sigma-4-0-mm-3D_gldm_DependenceNonUniformityNormalized', 'log-sigma-3-0-mm-3D_firstorder_Skewness', 'log-sigma-2-0-mm-3D_firstorder_MeanAbsoluteDeviation', 'log-sigma-4-0-mm-3D_firstorder_MeanAbsoluteDeviation', 'original_glrlm_RunLengthNonUniformity', 'original_glrlm_GrayLevelNonUniformity', 'log-sigma-2-0-mm-3D_firstorder_InterquartileRange', 'wavelet-LHL_glcm_Contrast', 'log-sigma-1-0-mm-3D_glrlm_RunEntropy', 'log-sigma-5-0-mm-3D_firstorder_Kurtosis', 'log-sigma-4-0-mm-3D_glrlm_RunEntropy', 'wavelet-LLH_glrlm_GrayLevelNonUniformity', 'original_shape_VoxelVolume', 'log-sigma-2-0-mm-3D_glrlm_LongRunLowGrayLevelEmphasis', 'log-sigma-5-0-mm-3D_glrlm_RunVariance', 'original_glrlm_RunVariance', 'log-sigma-4-0-mm-3D_firstorder_InterquartileRange', 'wavelet-LLL_firstorder_Minimum', 'wavelet-LHL_glcm_Id', 'original_shape_Elongation', 'log-sigma-2-0-mm-3D_firstorder_Variance', 'log-sigma-5-0-mm-3D_firstorder_Variance', 'log-sigma-3-0-mm-3D_glcm_Idn', 'wavelet-HLL_gldm_DependenceNonUniformity', 'log-sigma-5-0-mm-3D_glrlm_LongRunEmphasis', 'log-sigma-3-0-mm-3D_glszm_SizeZoneNonUniformityNormalized', 'wavelet-HHH_glrlm_LongRunEmphasis', 'log-sigma-1-0-mm-3D_firstorder_Range', 'log-sigma-3-0-mm-3D_glszm_ZoneEntropy', 'wavelet-LHL_firstorder_TotalEnergy', 'log-sigma-3-0-mm-3D_glrlm_RunLengthNonUniformity', 'log-sigma-1-0-mm-3D_firstorder_TotalEnergy', 'wavelet-HLL_firstorder_RobustMeanAbsoluteDeviation', 'log-sigma-5-0-mm-3D_glrlm_RunLengthNonUniformityNormalized', 'log-sigma-2-0-mm-3D_firstorder_RootMeanSquared', 'wavelet-LHL_glrlm_ShortRunLowGrayLevelEmphasis', 'wavelet-LLH_firstorder_TotalEnergy', 'wavelet-HHH_gldm_DependenceNonUniformity', 'log-sigma-3-0-mm-3D_firstorder_InterquartileRange', 'wavelet-LLH_glcm_DifferenceAverage', 'log-sigma-5-0-mm-3D_glrlm_LongRunHighGrayLevelEmphasis', 'wavelet-LLL_firstorder_10Percentile', 'original_firstorder_10Percentile', 'log-sigma-5-0-mm-3D_gldm_SmallDependenceLowGrayLevelEmphasis', 'log-sigma-1-0-mm-3D_glrlm_LongRunHighGrayLevelEmphasis', 'wavelet-LLL_glrlm_RunLengthNonUniformity', 'log-sigma-3-0-mm-3D_glrlm_LongRunEmphasis', 'log-sigma-4-0-mm-3D_firstorder_Maximum', 'log-sigma-5-0-mm-3D_firstorder_Entropy', 'log-sigma-5-0-mm-3D_firstorder_Maximum', 'log-sigma-5-0-mm-3D_glrlm_RunLengthNonUniformity', 'log-sigma-1-0-mm-3D_firstorder_Kurtosis', 'log-sigma-1-0-mm-3D_firstorder_RobustMeanAbsoluteDeviation', 'log-sigma-5-0-mm-3D_gldm_LargeDependenceHighGrayLevelEmphasis', 'log-sigma-1-0-mm-3D_firstorder_InterquartileRange', 'log-sigma-4-0-mm-3D_glszm_ZonePercentage', 'wavelet-LLH_glcm_Idn', 'original_glrlm_ShortRunEmphasis', 'log-sigma-1-0-mm-3D_firstorder_Variance', 'log-sigma-1-0-mm-3D_firstorder_Skewness', 'log-sigma-3-0-mm-3D_firstorder_10Percentile', 'log-sigma-5-0-mm-3D_glszm_GrayLevelNonUniformityNormalized', 'log-sigma-1-0-mm-3D_gldm_LargeDependenceHighGrayLevelEmphasis', 'log-sigma-2-0-mm-3D_gldm_DependenceNonUniformity', 'wavelet-LLH_firstorder_10Percentile', 'log-sigma-3-0-mm-3D_firstorder_RobustMeanAbsoluteDeviation', 'wavelet-LHL_glcm_Idmn', 'log-sigma-5-0-mm-3D_glcm_ClusterTendency', 'log-sigma-3-0-mm-3D_glrlm_RunEntropy', 'wavelet-LLH_firstorder_Maximum', 'log-sigma-1-0-mm-3D_firstorder_10Percentile', 'original_shape_MinorAxisLength', 'log-sigma-3-0-mm-3D_glcm_ClusterShade', 'log-sigma-5-0-mm-3D_firstorder_Range', 'log-sigma-5-0-mm-3D_glcm_Id', 'log-sigma-5-0-mm-3D_firstorder_Minimum', 'wavelet-LLL_gldm_DependenceVariance', 'log-sigma-4-0-mm-3D_glszm_GrayLevelNonUniformityNormalized', 'original_glcm_ClusterTendency', 'log-sigma-4-0-mm-3D_glcm_ClusterShade', 'original_glcm_JointEntropy', 'log-sigma-4-0-mm-3D_firstorder_Range', 'log-sigma-5-0-mm-3D_glrlm_LongRunLowGrayLevelEmphasis', 'log-sigma-5-0-mm-3D_glcm_Imc2', 'log-sigma-5-0-mm-3D_firstorder_RobustMeanAbsoluteDeviation', 'original_shape_Flatness', 'log-sigma-2-0-mm-3D_firstorder_RobustMeanAbsoluteDeviation', 'wavelet-LHL_glcm_Idn', 'log-sigma-5-0-mm-3D_glrlm_RunEntropy', 'log-sigma-2-0-mm-3D_firstorder_Maximum', 'log-sigma-5-0-mm-3D_firstorder_Mean', 'log-sigma-2-0-mm-3D_firstorder_Median', 'log-sigma-4-0-mm-3D_glszm_GrayLevelNonUniformity', 'wavelet-LLL_gldm_DependenceNonUniformityNormalized', 'log-sigma-4-0-mm-3D_glszm_SmallAreaEmphasis', 'original_shape_Maximum2DDiameterColumn', 'log-sigma-1-0-mm-3D_firstorder_RootMeanSquared', 'log-sigma-2-0-mm-3D_glszm_SmallAreaHighGrayLevelEmphasis', 'wavelet-LLL_firstorder_90Percentile', 'log-sigma-4-0-mm-3D_glcm_Idmn', 'log-sigma-1-0-mm-3D_gldm_GrayLevelVariance', 'log-sigma-2-0-mm-3D_glcm_JointEntropy', 'wavelet-LHL_glrlm_RunLengthNonUniformity', 'log-sigma-3-0-mm-3D_glcm_Imc1', 'log-sigma-2-0-mm-3D_firstorder_Minimum', 'log-sigma-5-0-mm-3D_firstorder_RootMeanSquared', 'log-sigma-5-0-mm-3D_firstorder_TotalEnergy', 'log-sigma-5-0-mm-3D_glszm_GrayLevelVariance', 'log-sigma-1-0-mm-3D_firstorder_Maximum', 'log-sigma-3-0-mm-3D_glcm_DifferenceVariance', 'log-sigma-4-0-mm-3D_glcm_Idn', 'log-sigma-2-0-mm-3D_gldm_DependenceVariance', 'log-sigma-3-0-mm-3D_firstorder_Mean', 'log-sigma-3-0-mm-3D_glcm_Imc2', 'log-sigma-5-0-mm-3D_gldm_SmallDependenceHighGrayLevelEmphasis', 'log-sigma-5-0-mm-3D_gldm_DependenceNonUniformityNormalized', 'wavelet-HLL_firstorder_10Percentile', 'log-sigma-3-0-mm-3D_gldm_DependenceVariance', 'log-sigma-4-0-mm-3D_glszm_LargeAreaHighGrayLevelEmphasis', 'log-sigma-5-0-mm-3D_firstorder_MeanAbsoluteDeviation', 'wavelet-HLL_glcm_SumSquares', 'log-sigma-2-0-mm-3D_glcm_DifferenceAverage', 'log-sigma-3-0-mm-3D_gldm_LargeDependenceHighGrayLevelEmphasis', 'wavelet-LLH_firstorder_Minimum', 'wavelet-HLL_firstorder_Minimum', 'log-sigma-1-0-mm-3D_firstorder_Entropy', 'log-sigma-3-0-mm-3D_glcm_DifferenceEntropy', 'log-sigma-1-0-mm-3D_glrlm_GrayLevelVariance', 'wavelet-LLL_glrlm_LongRunEmphasis', 'log-sigma-4-0-mm-3D_glszm_SmallAreaHighGrayLevelEmphasis', 'wavelet-LLL_firstorder_Kurtosis', 'log-sigma-4-0-mm-3D_glrlm_LongRunLowGrayLevelEmphasis', 'wavelet-LHL_glcm_Idm', 'log-sigma-5-0-mm-3D_glrlm_GrayLevelNonUniformityNormalized', 'original_firstorder_Minimum', 'log-sigma-3-0-mm-3D_firstorder_Kurtosis', 'original_firstorder_Variance', 'wavelet-HHL_gldm_GrayLevelNonUniformity', 'wavelet-LHL_glcm_Correlation', 'log-sigma-1-0-mm-3D_firstorder_Minimum', 'wavelet-LLL_gldm_LargeDependenceHighGrayLevelEmphasis', 'log-sigma-4-0-mm-3D_firstorder_Uniformity', 'log-sigma-1-0-mm-3D_glrlm_ShortRunLowGrayLevelEmphasis', 'wavelet-LHL_glcm_SumSquares', 'wavelet-LLH_glcm_Idm', 'wavelet-LLH_glcm_InverseVariance', 'wavelet-HLL_firstorder_MeanAbsoluteDeviation', 'original_firstorder_Energy', 'log-sigma-1-0-mm-3D_glcm_SumEntropy', 'wavelet-LLL_glcm_Imc2', 'log-sigma-1-0-mm-3D_glcm_Imc1', 'original_firstorder_Mean', 'log-sigma-5-0-mm-3D_glcm_ClusterProminence', 'log-sigma-5-0-mm-3D_glcm_Imc1', 'wavelet-HHH_firstorder_TotalEnergy', 'wavelet-LLL_firstorder_Variance', 'log-sigma-4-0-mm-3D_glcm_InverseVariance', 'log-sigma-4-0-mm-3D_firstorder_10Percentile', 'wavelet-LLH_glcm_Imc1', 'log-sigma-2-0-mm-3D_glszm_LargeAreaLowGrayLevelEmphasis', 'wavelet-HHL_glrlm_LongRunLowGrayLevelEmphasis', 'log-sigma-2-0-mm-3D_glcm_Correlation', 'log-sigma-2-0-mm-3D_glrlm_RunPercentage', 'log-sigma-4-0-mm-3D_firstorder_Minimum', 'original_firstorder_Range', 'log-sigma-1-0-mm-3D_gldm_GrayLevelNonUniformity', 'log-sigma-3-0-mm-3D_firstorder_MeanAbsoluteDeviation', 'wavelet-LLH_glcm_Correlation', 'log-sigma-2-0-mm-3D_gldm_SmallDependenceEmphasis', 'log-sigma-4-0-mm-3D_gldm_DependenceNonUniformity', 'log-sigma-3-0-mm-3D_firstorder_RootMeanSquared', 'log-sigma-2-0-mm-3D_firstorder_Energy', 'log-sigma-4-0-mm-3D_glrlm_RunPercentage', 'log-sigma-4-0-mm-3D_glcm_JointEnergy', 'log-sigma-4-0-mm-3D_firstorder_RobustMeanAbsoluteDeviation']
wavelet=0
Log=0
Other=0
for fetuare in RM_RF_model_200_features[:100]:
    if fetuare.split('-')[0]=='wavelet':
        wavelet=wavelet+1
    elif fetuare.split('-')[0]=='log':
        Log=Log+1
    else:
        print(fetuare)
print(wavelet,Log,Other)




original_glrlm_RunEntropy
original_glrlm_RunLengthNonUniformity
original_glrlm_GrayLevelNonUniformity
original_shape_VoxelVolume
original_glrlm_RunVariance
original_shape_Elongation
original_firstorder_10Percentile
original_glrlm_ShortRunEmphasis
original_shape_MinorAxisLength
21 70 0
